In [ ]:
import json

import pandas as pd
import seaborn as sns

In [ ]:
with open("data/arrivals_dump.json") as f:
    lines = f.readlines()
data = json.loads("\n".join([line[:-1] if line.endswith("+") else line for line in ("[" + "".join(lines[3:-2])).split("\n")]))
frame = pd.DataFrame(data)
frame.set_index("id", inplace=True)
frame.reset_index(inplace=True)
datetime_cols = ["generatedOn", "arrivalAt", "createdAt", "updatedAt"]
for col in datetime_cols:
    frame[col] = pd.to_datetime(frame[col])
frame.info()

In [ ]:
sns.set_theme()
sns.relplot(data=frame, x="generatedOn", y="arrivalAt")